# **Retrievers**

## **What's Covered?**
- Introduction to Retrievers
    - Specify Top k
    - Specify Top k and Search Type
    - Maximum Marginal Relevance Retrieval
    - Similarity Score Threshold Retrieval
- Building an End-to-End RAG Chain
    - Step 1: Initialize the Chroma DB Connection
    - Step 2: Create a Retriever Object
    - Step 3: Initialize a Chat Prompt Template
    - Step 4: Initialize a Generator (i.e. Chat Model)
    - Step 5: Initialize a Output Parser
    - Step 6: Define a RAG Chain
    - Step 7: Invoke the Chain

Introduction to Retrievers
There are times when we need to pass in Vector Stores as retriever objects, which can be easily done via a as_retriever() method call.

A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

Retrievers accept a string query as input and return a list of Document's as output.

We can specify search type implemented by a vector store, like similarity and MMR (i.e. Maximum marginal relevance retrieval), to query the texts in the vector store.

Specify Top k
retriever = db_connection.as_retriever(search_kwargs={"k": 3})
Specify Top k and Search Type
retriever = db_connection.as_retriever(search_type="similarity", search_kwargs={"k": 3})
Maximum Marginal Relevance Retrieval
retriever = db_connection.as_retriever(search_type="mmr")
Similarity Score Threshold Retrieval
Apply a cutoff or a threshold such that any document which is below the cutoff is not returned.
retriever = db_connection.as_retriever(
  search_type="similarity_score_threshold",
  search_kwargs={"k": 3, "score_threshold": 0.5}
)

In [1]:
from google.colab import userdata
OPENAI_API_Key = userdata.get('OpenAIAPIKey')

In [2]:
!pip install langchain

In [3]:
!pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.0 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.13
    Uninstalling langchain-core-0.3.13:
      Successfully uninstalled langchain-core-0.3.13


In [4]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.7/615.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.2 MB/s eta 0:00:0

# Step-1 Initialize an Embedding Model

In [5]:
# We are just loading OpenAIEmbeddings

from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(openai_api_key =OPENAI_API_Key)

# Step 2- Initialize the ChromDB Connecction

In [6]:
pip install langchain-chroma

In [7]:
import chromadb

In [20]:
from langchain_chroma import Chroma

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created

db = Chroma(collection_name = "vector_database",
            embedding_function = embedding_model,
            persist_directory = "./chroma_db_")


In [21]:
db

In [22]:
# We can check the already existing values

print(len(db.get()["ids"]))

0


In [23]:
# Converting Chroma db connection to Retriever Object

retriever = db.as_retriever(search_type = 'similarity',search_kwargs = {"k":3})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


In [24]:
query = "What is there on Julie vs Rachels List?"

results = retriever.invoke(query)

In [25]:
results

[]

## **Building an End-to-End RAG Chain**

**Step 1: Initialize the Chroma DB Connection**  
**Step 2: Create a Retriever Object**   
**Step 3: Initialize a Chat Prompt Template**  
**Step 4: Initialize a Generator (i.e. Chat Model)**  
**Step 5: Initialize a Output Parser**   
**Step 6: Define a RAG Chain**  
**Step 7: Invoke the Chain**

In [27]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.4
    Uninstalling langchain-0.3.4:
      Successfully uninstalled langchain-0.3.4


In [28]:
# Step 1: Initialize the Chroma DB Connection

from langchain.vectorstores import Chroma

#from langchain_chroma import Chroma

# Initialize the database connection
# If database exist, it will connect with the collection_name and persist_directory
# Otherwise a new collection will be created
db = Chroma(collection_name="vector_database",
            embedding_function=embedding_model,
            persist_directory="./chroma_db_")

# We can check the already existing values
print(len(db.get()["ids"]))

0


<ipython-input-28-dd3b00b3bb44>:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(collection_name="vector_database",


In [29]:
# Step 2: Create a Retriever Object

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [30]:
# Step 3: Initialize a Chat Prompt Template

from langchain_core.prompts import ChatPromptTemplate

PROMPT_TEMPLATE = """
Answer the question based only on the following context:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Don’t justify your answers.
Don’t give information not mentioned in the CONTEXT INFORMATION.
Do not say "according to the context" or "mentioned in the context" or similar.
"""

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [32]:
# Step 4: Initialize a Generator (i.e. Chat Model)

from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(openai_api_key =OPENAI_API_Key)

In [33]:
# Step 5: Initialize a Output Parser

from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
# Step 6: Define a RAG Chain

from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = {"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt_template | chat_model | parser

In [35]:
# Invoke the Chain

query = 'Who is Rachem?'

rag_chain.invoke(query)

'Rachem is the name of a person mentioned in the context.'

In [36]:
# Invoke the Chain

query = 'What is there on the List comparing Rachel and Julie?'

rag_chain.invoke(query)

'The List compares Rachel and Julie based on their academic achievements and extracurricular activities.'